# Text Generation using LSTM

In [1]:
skip_training = False   # You can set it to True if you want to run inference on your trained model.

### Import the necessary libraries

In [2]:
import random
import re

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [3]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Load and Preprocess the Text Dataset

We will be using *Alice's Adventures in Wonderland* by Lewis Carroll as our dataset. You can download it from [Project Gutenberg](https://www.gutenberg.org/):

[Alice's Adventures in Wonderland by Lewis Carroll (Project Gutenberg Page)](https://www.gutenberg.org/ebooks/11) \
[Direct Text File Download](https://www.gutenberg.org/files/11/11-0.txt)

I've chosen Alice's Adventures in Wonderland as a relatively small text to make training still manageable on a CPU.

#### Load the Dataset

We start by loading the text dataset into Python. The dataset should be a plain text file. The first step is to load and inspect a small portion of the raw text to understand its structure to identify any unwanted metadata or special characters that should be removed during preprocessing.

In [ ]:
txt_path = 'alice.txt'

In [ ]:

with open(txt_path, 'r', encoding = 'utf-8') as file:
    raw_text = file.read()

print('===First 1500 characters before any processing:\n\n')
print(raw_text[:1500])

print('\n\n\n===Ending characters before any processing:\n')
print(raw_text[-19000:-17000])

===First 1500 characters before any processing:


﻿The Project Gutenberg eBook of Alice's Adventures in Wonderland
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: Alice's Adventures in Wonderland

Author: Lewis Carroll

Release date: June 27, 2008 [eBook #11]
                Most recently updated: October 21, 2024

Language: English

Credits: Arthur DiBianca and David Widger


*** START OF THE PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***
[Illustration]




Alice’s Adventures in Wonderland

by Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0

Contents

 CHAPTER I.     D

#### Remove Metadata and Focus on the Main Text


In [ ]:
# For this example, I'm removing everything before 'CHAPTER I.\nDown the Rabbit-Hole'
# and after the end marker
start_index = raw_text.find('CHAPTER I.\nDown the Rabbit-Hole')

end_index = raw_text.find('*** END OF THE PROJECT GUTENBERG') # closing markers of Project Gutenberg

trimmed_text = raw_text[start_index:end_index]

print('===Text after removing metadata:\n')
print(trimmed_text[:1500])

===Text after removing metadata:

CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so _very_ remarkable in that; nor did Alice think it
so _very_ much out of the way to hear the Rabbit say to itself, “Oh
dear! Oh dear! I shall be late!” (when she thought it over afterwards,
it occurred to her that she ought to have wondered at this, but at the
time it all seemed quite natural); but when the Rabbit a

#### Clean the Text

In [ ]:
def preprocess_text(text):
   
    lowered_text=text.lower()
    lowered_text = re.sub(r'[^a-z0-9\s]', ' ', lowered_text)
    cleaned_text = re.sub(r'\s+', ' ', lowered_text)


    return cleaned_text

cleaned_text = preprocess_text(trimmed_text)
print('Text after cleaning and converting to lowercase:\n')
print(cleaned_text[:1000])


Text after cleaning and converting to lowercase:

chapter i down the rabbit hole alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought alice without pictures or conversations so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her there was nothing so very remarkable in that nor did alice think it so very much out of the way to hear the rabbit say to itself oh dear oh dear i shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite natural but when the rabbit actually took a watch 

### Character-Level Encoding



#### Character-Level Vocabulary

In [ ]:
def create_char_mappings(cleaned_text):
    
    dico = sorted(set(cleaned_text))
    char_to_int = {char: idx for idx, char in enumerate(dico)}
    int_to_char = {idx: char for char, idx in char_to_int.items()}

    return char_to_int, int_to_char

char_to_int, int_to_char = create_char_mappings(cleaned_text)
print('Character to Integer Mapping:')
for char, idx in list(char_to_int.items()):
    print(f"'{char}' : {idx}")

Character to Integer Mapping:
' ' : 0
'a' : 1
'b' : 2
'c' : 3
'd' : 4
'e' : 5
'f' : 6
'g' : 7
'h' : 8
'i' : 9
'j' : 10
'k' : 11
'l' : 12
'm' : 13
'n' : 14
'o' : 15
'p' : 16
'q' : 17
'r' : 18
's' : 19
't' : 20
'u' : 21
'v' : 22
'w' : 23
'x' : 24
'y' : 25
'z' : 26


#### Encode the Text into Integers

In [ ]:
def encode_text(cleaned_text, char_to_int):
    
    encoded_chars=[char_to_int[n] for n in cleaned_text]

    return encoded_chars

encoded_chars = encode_text(cleaned_text, char_to_int)
print(f"Total encoded characters: {len(encoded_chars)}")
print('First 100 encoded characters:')
print(encoded_chars[:100])

Total encoded characters: 135002
First 100 encoded characters:
[3, 8, 1, 16, 20, 5, 18, 0, 9, 0, 4, 15, 23, 14, 0, 20, 8, 5, 0, 18, 1, 2, 2, 9, 20, 0, 8, 15, 12, 5, 0, 1, 12, 9, 3, 5, 0, 23, 1, 19, 0, 2, 5, 7, 9, 14, 14, 9, 14, 7, 0, 20, 15, 0, 7, 5, 20, 0, 22, 5, 18, 25, 0, 20, 9, 18, 5, 4, 0, 15, 6, 0, 19, 9, 20, 20, 9, 14, 7, 0, 2, 25, 0, 8, 5, 18, 0, 19, 9, 19, 20, 5, 18, 0, 15, 14, 0, 20, 8, 5]


### Batch Generation for Training


In [ ]:
def get_batches(arr, batch_size, seq_length, step_size=None):
    
    if step_size is None:
        step_size = seq_length
    n=len(arr)
    numberofbatches = (n-1 - seq_length) // (step_size * batch_size)
    x_batches=[]
    y_batches=[]
    for i in range (numberofbatches):
      x_batch=[]
      y_batch=[]
      for k in range(batch_size):
        start=(step_size * batch_size) * i + k * step_size
        end=start+seq_length

        x_batch.append(arr[start:end])
        y_batch.append(arr[start+1:end+1])
      x_batches.append(x_batch)
      y_batches.append(y_batch)
    x_batches=np.array(x_batches)
    y_batches=np.array(y_batches)



    return x_batches, y_batches

In [34]:
# Display for y shift and  step_size
def display_batch_generation(arr, char_to_int, int_to_char):
    batch_size, seq_length, step_size = 8, 10, 5  # Setting step_size for overlap between sequences

    x_batches, y_batches = get_batches(arr, batch_size, seq_length, step_size)

    # Display batch number 10
    x_chars = ''.join([int_to_char[idx] for idx in x_batches[10][0]])
    y_chars = ''.join([int_to_char[idx] for idx in y_batches[10][0]])

    print('='*50)
    print('Displaying a Single Batch')
    print('='*50)
    for i in range(batch_size):
        x_chars = ''.join([int_to_char[idx] for idx in x_batches[10][i]])
        y_chars = ''.join([int_to_char[idx] for idx in y_batches[10][i]])

        print(f"[{x_chars}]  -->  [{y_chars}]")
    print('='*50)
display_batch_generation(encoded_chars, char_to_int, int_to_char )

Displaying a Single Batch
[made her f]  -->  [ade her fe]
[her feel v]  -->  [er feel ve]
[eel very s]  -->  [el very sl]
[ery sleepy]  -->  [ry sleepy ]
[leepy and ]  -->  [eepy and s]
[ and stupi]  -->  [and stupid]
[stupid whe]  -->  [tupid whet]
[d whether ]  -->  [ whether t]


### Define the Character-Level LSTM Model 

[nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
[nn.Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html)
[nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)


In [ ]:
class CharLSTM(nn.Module):
    

    def __init__(self, num_layers, input_dim, hidden_dim, output_dim, dropout_prob):
        
        super(CharLSTM, self).__init__()

        # Save hidden dimension and number of layers for hidden state initialization
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            dropout=dropout_prob,
                            batch_first=True)

        self.dropout = nn.Dropout(dropout_prob)

        self.fc = nn.Linear(hidden_dim, output_dim)


    def forward(self, x, hidden):
        
        out, (h, c) = self.lstm(x, hidden)
        out = self.dropout(out)
        out = self.fc(out)


        # Return the final output and the updated hidden states
        return out, (h, c)

    def init_hidden(self, batch_size):
        
    
        device = next(self.parameters()).device

        # Initialize hidden state (h0) and cell state (c0) to zeros
        
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim, device=device)


        return (h0, c0)

#### Train the Model 
    [Adam_optimizer](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html)
    [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [ ]:
def train(model, encoded_chars, vocab_size, num_epochs, batch_size,
          seq_length, step_size, learning_rate, save_path=None, verbose=True):

    model.train()  
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  
    criterion = nn.CrossEntropyLoss()

    # Prepare batches
    x_batches, y_batches = get_batches(encoded_chars, batch_size, seq_length, step_size)
    num_batches = len(x_batches)

    for epoch in range(num_epochs):
        total_loss = 0
        # Progress bar for the current epoch
        batch_loader = tqdm(zip(x_batches, y_batches), total=num_batches,
                            leave=True, desc=f'Epoch {epoch+1}/{num_epochs}')

        # Initialize hidden states for both LSTM layers
        hidden=model.init_hidden(batch_size)


        for x, y in batch_loader:
            x = torch.as_tensor(x, dtype=torch.long).to(device)
            y = torch.as_tensor(y, dtype=torch.long).to(device) # target

            
            x = F.one_hot(x, num_classes=vocab_size).float()

            hidden = tuple(h.detach() for h in hidden)
            hidden = tuple(h.to(device) for h in hidden)

            logits, hidden = model(x, hidden)

            logits = logits.reshape(-1, vocab_size)
            y = y.reshape(-1)
            optimizer.zero_grad()
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Print the average loss for the current epoch
        if verbose:
            print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / num_batches:.4f}')

       
        if save_path:
            torch.save(model.state_dict(), save_path)
            print(f'Your trained model at epoch {epoch} is saved successfully!')

    return total_loss / num_batches

#### Model Initialization

In [59]:
hidden_dim = 400
dropout_prob=0.1
num_layers=2
vocab_size = len(char_to_int)
model = CharLSTM(num_layers, vocab_size, hidden_dim, vocab_size, dropout_prob)
model = model.to(device)
print(model)

CharLSTM(
  (lstm): LSTM(27, 400, num_layers=2, batch_first=True, dropout=0.1)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=400, out_features=27, bias=True)
)


In [ ]:
num_epochs = 50
batch_size = 50
seq_length=100
step_size=100
learning_rate=0.001


In [61]:
if not skip_training:
    loss = train(
        model=model,
        encoded_chars=encoded_chars,
        vocab_size=vocab_size,
        num_epochs=num_epochs,
        batch_size=batch_size,
        seq_length=seq_length,
        step_size=step_size,
        learning_rate=learning_rate,
        save_path='best_model.pth'
    )
else:
    model.load_state_dict(torch.load('best_model.pth', weights_only=False, map_location=device))
    print('Loaded weights from your saved model successfully!')

Epoch 1/50: 100%|██████████| 26/26 [00:01<00:00, 21.72it/s]


Epoch 1/50, Loss: 2.9128
Your trained model at epoch 0 is saved successfully!


Epoch 2/50: 100%|██████████| 26/26 [00:00<00:00, 31.83it/s]


Epoch 2/50, Loss: 2.8064
Your trained model at epoch 1 is saved successfully!


Epoch 3/50: 100%|██████████| 26/26 [00:00<00:00, 31.55it/s]


...

Epoch 48/50, Loss: 0.8448
Your trained model at epoch 47 is saved successfully!


Epoch 49/50: 100%|██████████| 26/26 [00:00<00:00, 32.88it/s]


Epoch 49/50, Loss: 0.8264
Your trained model at epoch 48 is saved successfully!


Epoch 50/50: 100%|██████████| 26/26 [00:00<00:00, 33.03it/s]

Epoch 50/50, Loss: 0.8013
Your trained model at epoch 49 is saved successfully!


### Text Generation

In [ ]:
def generate_text(model, start_str, char_to_int, int_to_char, vocab_size, predict_len=100, temperature=1.0):
    
    model.eval()  

    
    input_seq = [char_to_int[char] for char in start_str]
    input_seq = torch.tensor(input_seq).long().to(device).unsqueeze(0)  

    hidden = model.init_hidden(1)  # Batch size of 1 for generating text

    generated_text = start_str

    with torch.no_grad():  
        for _ in range(predict_len):

            
            x= F.one_hot(input_seq, num_classes=vocab_size).float()
            output, hidden = model(x, hidden)
            output = output[:, -1, :]
            output=output/temperature


            # Convert output to probabilities using softmax
            probabilities = F.softmax(output, dim=-1).detach().cpu().numpy()

            # Randomly sample based on the output probabilities
            next_char_index = np.random.choice(range(vocab_size), p=probabilities.ravel())

            # Add the predicted character to the generated text
            next_char = int_to_char[next_char_index]
            generated_text += next_char

            # Update the input sequence
            input_seq = torch.cat([input_seq[:, 1:], torch.tensor([[next_char_index]]).to(device)], dim=1)

    return generated_text

In [72]:
start_str = 'we re all '
predict_len = 1000
temperature = 0.5
generated_text = generate_text(model,
                               start_str,
                               char_to_int,
                               int_to_char,
                               vocab_size,
                               predict_len=predict_len,
                               temperature=temperature)
print(generated_text)

we re all of the whole plose to have her and the words said alice i didn t said the mock turtle these was no wish of i don t go no sure of the only would be of her hand and every onine had been parsing at the players all i ve to to do this time of the lobster to begin with you mo not i ll get me to shin the exceponere i was she what s the jury or eagle with alice was a little marked alice indorainy not to be at all say the mock turtle they were now and then to do so she began with a suppy said alice went on without to the three gardeners in her hand at the mock turtle they were now only the other but the mock turtle of the book the officed the end of the beginning to the blay and hand and they would no too mo no time so saving all the three gardeners so she began in a lougn and began in a great quistle theie watching the end of the same with a replied so alice replied the queen so said the other but the top of the execution when they growing so she began no began which she roon and no 

While LSTMs grasp the grammar well, they struggle with long-term coherence compared to modern Transformer architectures. This project demonstrates the fundamentals of sequence modeling.